In [14]:
import datetime as dt
import ast
import json
from collections import Counter
import os

import numpy as np
import pandas as pd
import string
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Flatten, Dense, concatenate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn import preprocessing
import statsmodels.api as sm

In [15]:
df = pd.read_csv('new_cars.csv')
print(df.dtypes)

link                        object
listing_title               object
listing_mileage              int64
primary_price                int64
deal_gauge                  object
exterior_color              object
interior_color              object
drivetrain                  object
mpg                         object
fuel_type                   object
transmission                object
engine                      object
vin                         object
stock_number                object
vehicle_history             object
seller_name                 object
price_history               object
Accidents or damage           bool
1-owner vehicle             object
Personal use only           object
Open recall                   bool
Clean title                 object
year                         int64
make                        object
model                       object
listed_date                 object
price_change               float64
price_change_percentage    float64
Cylinders           

In [16]:
### Drop unncessary columns
columns_to_drop = ['link', 'stock_number', 'vin', 'deal_gauge', 'exterior_color', 'mpg', 'fuel_type', 'engine', 'price_history', 'vehicle_history', 'listing_title', 'interior_color']
df = df.drop(columns=columns_to_drop)

In [17]:
# Drop 'model' since it has too many unique categorical values
df.drop(columns=['model'], inplace=True)

In [18]:
### Convert data types
boolean_columns = ['1-owner vehicle', 'Personal use only', 'Clean title']

for column in boolean_columns:
    df[column] = df[column].map({'Yes': True, 'No': False}).astype(bool)

In [19]:
# Check for NAs
missing_counts = df.isna().sum()
print(missing_counts[missing_counts > 0])

seller_name                  44
listed_date                1522
price_change               1522
price_change_percentage    1522
Cylinders                    73
Liters                      105
exterior_color_1            227
exterior_color_2           5989
interior_color_1            539
interior_color_2           5796
low mpg                     424
high mpg                    424
dtype: int64


In [20]:
# Drop rows with a low number of missing values
columns_with_missing_values = ['seller_name', 'Cylinders', 'Liters', 'exterior_color_1', 'low mpg', 'high mpg', 'interior_color_1']
df = df.dropna(subset=columns_with_missing_values)

# Drop exterior_color_2 since it only has 12 values
df = df.drop(columns=['exterior_color_2'])
# Drop interior_color_2 since it only has 203 values
df = df.drop(columns=['interior_color_2'])

# Drop listed_date, price_change, price_change_percentage because they account for 25% of data, 
# and they are not correlated with price in the correlation matrix.
columns_to_drop = ['listed_date', 'price_change', 'price_change_percentage']
df = df.drop(columns=columns_to_drop)

In [21]:
### Code related to price_history, which we decided to remove

# Convert listed_date to Days Avaliable
# current_date = pd.Timestamp(dt.datetime.now().date())
# df['listed_date'] = pd.to_datetime(df['listed_date'])
# df['Days Avaliable'] = (current_date - df['listed_date']).dt.days
# df = df.drop('listed_date', axis=1)
# df.head()

# # Drop missing price history data. ~25% of data.
# # columns_with_missing_values = ['price_change', 'price_change_percentage']
# # df = df.dropna(subset=columns_with_missing_values)

# # Drop 'price_change' and 'price_change_percentage' to avoid data leakage
# columns_to_drop = ['price_change', 'price_change_percentage']
# df = df.drop(columns=columns_to_drop)

In [22]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,year,make,Cylinders,Liters,Boosted,exterior_color_1,interior_color_1,low mpg,high mpg
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,False,True,False,True,True,2018,Hyundai,4.0,2.4,False,blue,beige,21.0,27.0
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,False,True,2017,RAM,8.0,5.7,False,silver,black,16.0,23.0
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,False,True,False,True,2019,Nissan,4.0,1.6,False,black,gray,31.0,36.0
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,True,False,False,True,True,2019,RAM,8.0,5.7,False,gray,black,19.0,24.0
4,135385,26950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,True,True,2015,Jeep,6.0,3.6,False,blue,black,16.0,21.0


In [23]:
df.shape

(4801, 19)

### outcome variable: primary_price
### ocluded variables: price_change	price_change_percentage

## Categorical variables

In [24]:
categorical_columns = ['seller_name', 'drivetrain', 'transmission', 'make', 'model', 'exterior_color_1', 'interior_color_1']
categorical_columns.remove('model')

# Check how many unique values there are in categorical_columns
for column in categorical_columns:
    unique_values = df[column].nunique()
    print(f"Unique values for {column}:", unique_values)

# Models has a large number of unique values
# seller_name, transmission, make, model have a large number of unique values
# Consider embedding these instead of one-hot encode

Unique values for seller_name: 122
Unique values for drivetrain: 5
Unique values for transmission: 105
Unique values for make: 49
Unique values for exterior_color_1: 13
Unique values for interior_color_1: 11


In [25]:
X = df.drop('primary_price', axis=1)  # Features
y = df['primary_price']  # Target

In [26]:
print(X.dtypes)
print(y.dtype)

listing_mileage          int64
drivetrain              object
transmission            object
seller_name             object
Accidents or damage       bool
1-owner vehicle           bool
Personal use only         bool
Open recall               bool
Clean title               bool
year                     int64
make                    object
Cylinders              float64
Liters                 float64
Boosted                   bool
exterior_color_1        object
interior_color_1        object
low mpg                float64
high mpg               float64
dtype: object
int64


In [27]:
### One-hot Encode 
X = pd.get_dummies(X, columns=categorical_columns)

In [28]:
X.shape

(4801, 317)

In [29]:
# Check that X doesn't have object data
unsupported_types = []

for column in X.columns:
    dtype = X[column].dtype
    if dtype == 'object':
        unsupported_types.append((column, dtype))

print("Unsupported Data Types:")
for column, dtype in unsupported_types:
    print(f"{column}: {dtype}")


Unsupported Data Types:


In [30]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,year,make,Cylinders,Liters,Boosted,exterior_color_1,interior_color_1,low mpg,high mpg
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,False,True,False,True,True,2018,Hyundai,4.0,2.4,False,blue,beige,21.0,27.0
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,False,True,2017,RAM,8.0,5.7,False,silver,black,16.0,23.0
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,False,True,False,True,2019,Nissan,4.0,1.6,False,black,gray,31.0,36.0
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,True,False,False,True,True,2019,RAM,8.0,5.7,False,gray,black,19.0,24.0
4,135385,26950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,True,True,2015,Jeep,6.0,3.6,False,blue,black,16.0,21.0


## Split data into 80% training set and 20% validation set.

In [31]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert all columns to float
X_train = X_train.astype(float)
X_val = X_val.astype(float)
y_train = y_train.astype(float)
y_val = y_val.astype(float)

# Convert data to numpy array
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

In [32]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

X_train shape: (3840, 317)
X_val shape: (961, 317)
y_train shape: (3840,)
y_val shape: (961,)


In [33]:
# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train)
X_val_tensor = torch.Tensor(X_val)
y_val_tensor = torch.Tensor(y_val)

## Deep Neural Net

In [20]:
### --- First attempt ---

# class NeuralNetwork(nn.Module):
#     def __init__(self, input_size):
#         super(NeuralNetwork, self).__init__()
#         self.fc1 = nn.Linear(input_size, 256)
#         self.relu1 = nn.ReLU()
#         self.dropout1 = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(256, 128)
#         self.relu2 = nn.ReLU()
#         self.dropout2 = nn.Dropout(0.5)
#         self.fc3 = nn.Linear(128, 64)
#         self.relu3 = nn.ReLU()
#         self.dropout3 = nn.Dropout(0.5)
#         self.fc4 = nn.Linear(64, 1)  # Output layer for regression

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu1(x)
#         x = self.dropout1(x)
#         x = self.fc2(x)
#         x = self.relu2(x)
#         x = self.dropout2(x)
#         x = self.fc3(x)
#         x = self.relu3(x)
#         x = self.dropout3(x)
#         x = self.fc4(x)
#         return x

# # Create the model
# input_size = X_train_tensor.shape[1]
# model = NeuralNetwork(input_size)

# # Loss function for regression
# criterion = nn.MSELoss()

# # Optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [34]:
class NeuralNetwork(nn.Module):
    def __init__(self, hidden_layers, neurons_per_layer, dropout_rate):
        super(NeuralNetwork, self).__init__()
        self.fc_layers = nn.ModuleList()
        input_size = X_train_tensor.shape[1]

        # Add hidden layers
        for _ in range(hidden_layers):
            self.fc_layers.append(nn.Linear(input_size, neurons_per_layer))
            self.fc_layers.append(nn.ReLU())
            self.fc_layers.append(nn.Dropout(dropout_rate))
            input_size = neurons_per_layer

        # Output layer
        self.fc_layers.append(nn.Linear(input_size, 1))

    def forward(self, x):
        for layer in self.fc_layers:
            x = layer(x)
        return x

def create_model(hidden_layers, neurons_per_layer, dropout_rate):
    return NeuralNetwork(hidden_layers, neurons_per_layer, dropout_rate)

In [35]:
# Training loop function
def train_DNN(epochs, benchmark, model, optimizer):
    training_losses = []
    validation_losses = []
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        # Forward pass
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor.view(-1, 1))
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate training accuracy
        with torch.no_grad():
            train_outputs = model(X_train_tensor)
            train_loss = criterion(train_outputs, y_train_tensor.view(-1, 1))

        # Calculate validation accuracy
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor.view(-1, 1))
        
        # Record training loss and MAE
        if (epoch+1) % benchmark == 0:
            mae = torch.abs(outputs - y_train_tensor.view(-1, 1)).mean().item()
            val_mae = torch.abs(val_outputs - y_val_tensor.view(-1, 1)).mean().item()
            print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {train_loss.item()}, MAE: {int(mae)}')

            training_losses.append((epoch + 1, train_loss.item(), mae))
            validation_losses.append((epoch + 1, val_loss.item(), val_mae))

    return training_losses, validation_losses

In [36]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(model, X, y):
    # Forward pass
    outputs = model(X)
    
    # Calculate loss
    criterion = nn.MSELoss()
    loss = criterion(outputs, y.view(-1, 1))
    
    # Convert predictions and targets to numpy arrays
    y_pred = outputs.detach().numpy()
    y_true = y.numpy()
    
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    return loss.item(), mae, mse, rmse, r2

# After the training loop

# Evaluate the model on the training set
# train_loss, train_mae, train_mse, train_rmse, train_r2 = evaluate_model(model, criterion, X_train_tensor, y_train_tensor)

# # Evaluate the model on the validation set
# val_loss, val_mae, val_mse, val_rmse, val_r2 = evaluate_model(model, criterion, X_val_tensor, y_val_tensor)

# # Print the evaluation metrics
# print(f"Training:\nLoss: {train_loss:.4f}\nMAE: {train_mae:.4f}\nRMSE: {train_rmse:.4f}\nR^2: {train_r2:.4f}\n")
# print(f"Validation:\nLoss: {val_loss:.4f}\nMAE: {val_mae:.4f}\nRMSE: {val_rmse:.4f}\nR^2: {val_r2:.4f}")

In [303]:
### Iterate through parameters

# 3^4=81
# Took about and hour and a half to run
learning_rates = [0.01, 0.001, 0.0001]
learning_rates = [0.001, 0.0001]
hidden_layers_list = [1, 2, 3]
neurons_per_layer_list = [64, 128, 256]
dropout_rates = [0.1, 0.3, 0.5]

try:
    with open('results.json', 'r') as file:
        results = json.load(file)
except FileNotFoundError:
    results = []

for lr in learning_rates:
    for hidden_layers in hidden_layers_list:
        for neurons_per_layer in neurons_per_layer_list:
            for dropout_rate in dropout_rates:
                # Build and train the model with the current hyperparameters
                #print("--------------------------------------------------------------------------------------------------------")
                print(f"lr={lr}, hidden_layers={hidden_layers}, neurons_per_layer={neurons_per_layer}, dropout_rate={dropout_rate}")
                #model_save_path = f'models/{hidden_layers}_{neurons_per_layer}_{dropout_rate}'
                model = create_model(hidden_layers, neurons_per_layer, dropout_rate)
                optimizer = optim.Adam(model.parameters(), lr=lr)
                
                # Training loop 
                train_DNN(1000, 50, model, optimizer)
                #plot_loss_mae(training_loss_mae, model_save_path)

                # Evaluate the model on the training set
                train_loss, train_mae, train_mse, train_rmse, train_r2 = evaluate_model(model, X_train_tensor, y_train_tensor)
                #print(f"Training:\nLoss: {train_loss:.4f}\nMAE: {train_mae:.4f}\nRMSE: {train_rmse:.4f}\nR^2: {train_r2:.4f}\n")

                # Evaluate the model on the validation set
                val_loss, val_mae, val_mse, val_rmse, val_r2 = evaluate_model(model, X_val_tensor, y_val_tensor)
                #print(f"Validation:\nLoss: {val_loss:.4f}\nMAE: {val_mae:.4f}\nRMSE: {val_rmse:.4f}\nR^2: {val_r2:.4f}")

                result_entry = {
                    'lr': lr,
                    'hidden_layers': hidden_layers,
                    'neurons_per_layer': neurons_per_layer,
                    'dropout_rate': dropout_rate,
                    'train_loss': train_loss,
                    'train_mae': train_mae,
                    'train_mse': train_mse,
                    'train_rmse': train_rmse,
                    'train_r2': train_r2,
                    'val_loss': val_loss,
                    'val_mae': val_mae,
                    'val_mse': val_mse,
                    'val_rmse': val_rmse,
                    'val_r2': val_r2,
                    #'training_losses': training_losses
                }
                
                results.append(result_entry)

#with open('results.json', 'w') as file:
#    json.dump(results, file)

lr=0.001, hidden_layers=1, neurons_per_layer=64, dropout_rate=0.1
lr=0.001, hidden_layers=1, neurons_per_layer=64, dropout_rate=0.3
lr=0.001, hidden_layers=1, neurons_per_layer=64, dropout_rate=0.5
lr=0.001, hidden_layers=1, neurons_per_layer=128, dropout_rate=0.1
lr=0.001, hidden_layers=1, neurons_per_layer=128, dropout_rate=0.3
lr=0.001, hidden_layers=1, neurons_per_layer=128, dropout_rate=0.5
lr=0.001, hidden_layers=1, neurons_per_layer=256, dropout_rate=0.1
lr=0.001, hidden_layers=1, neurons_per_layer=256, dropout_rate=0.3
lr=0.001, hidden_layers=1, neurons_per_layer=256, dropout_rate=0.5
lr=0.001, hidden_layers=2, neurons_per_layer=64, dropout_rate=0.1
lr=0.001, hidden_layers=2, neurons_per_layer=64, dropout_rate=0.3
lr=0.001, hidden_layers=2, neurons_per_layer=64, dropout_rate=0.5
lr=0.001, hidden_layers=2, neurons_per_layer=128, dropout_rate=0.1
lr=0.001, hidden_layers=2, neurons_per_layer=128, dropout_rate=0.3
lr=0.001, hidden_layers=2, neurons_per_layer=128, dropout_rate=0.5
l

TypeError: Object of type float32 is not JSON serializable

In [307]:
#results
# Save to results.json
# Ran into a decoding error. Faster to just manually paste result printout into results.json than to bugfix.

In [310]:
### Check best hyperparameters:
# try:
#     with open('results.json', 'r') as file:
#         results = json.load(file)
# except json.JSONDecodeError as e:
#     print(f"Error decoding JSON: {e}")
# except FileNotFoundError:
#     print("No results.json file found.")


max_metrics = [
    'train_loss', 'val_loss',
    'train_mae', 'val_mae',
    'train_mse', 'val_mse',
    'train_rmse', 'val_rmse'
]
min_metrics = [
    'train_r2', 'val_r2'
]

# Initialize best_results for metrics where maximum is desired
best_results_max = {metric: float('-inf') for metric in max_metrics}
best_result_by_metric_max = {metric: None for metric in max_metrics}

# Initialize best_results for metrics where minimum is desired
best_results_min = {metric: float('inf') for metric in min_metrics}
best_result_by_metric_min = {metric: None for metric in min_metrics}

for result in results:
    # Update maximum metrics
    for metric in max_metrics:
        if result[metric] > best_results_max[metric]:
            best_results_max[metric] = result[metric]
            best_result_by_metric_max[metric] = result

    # Update minimum metrics
    for metric in min_metrics:
        if result[metric] < best_results_min[metric]:
            best_results_min[metric] = result[metric]
            best_result_by_metric_min[metric] = result

# Print the best result for each metric
for metric in max_metrics:
    print(f"Best {metric} Result:")
    print(best_result_by_metric_max[metric])
    print()

for metric in min_metrics:
    print(f"Best {metric} Result:")
    print(best_result_by_metric_min[metric])
    print()


### Best hyperparameters:
# 'lr': 0.0001                  8 / 8 results
# 'hidden_layers': 1            8 / 8 results
# 'neurons_per_layer': 64       8 / 8 results
# 'dropout_rate': 0.5           7 / 8 results

Best train_loss Result:
{'lr': 0.0001, 'hidden_layers': 1, 'neurons_per_layer': 64, 'dropout_rate': 0.5, 'train_loss': 1206575616.0, 'train_mae': 24048.715, 'train_mse': 1206575600.0, 'train_rmse': 34735.797, 'train_r2': -1.0821919102990356, 'val_loss': 1383415424.0, 'val_mae': 25250.082, 'val_mse': 1383415400.0, 'val_rmse': 37194.293, 'val_r2': -0.9946579521163981}

Best val_loss Result:
{'lr': 0.0001, 'hidden_layers': 1, 'neurons_per_layer': 64, 'dropout_rate': 0.5, 'train_loss': 1206575616.0, 'train_mae': 24048.715, 'train_mse': 1206575600.0, 'train_rmse': 34735.797, 'train_r2': -1.0821919102990356, 'val_loss': 1383415424.0, 'val_mae': 25250.082, 'val_mse': 1383415400.0, 'val_rmse': 37194.293, 'val_r2': -0.9946579521163981}

Best train_mae Result:
{'lr': 0.0001, 'hidden_layers': 1, 'neurons_per_layer': 64, 'dropout_rate': 0.5, 'train_loss': 1206575616.0, 'train_mae': 24048.715, 'train_mse': 1206575600.0, 'train_rmse': 34735.797, 'train_r2': -1.0821919102990356, 'val_loss': 138341542

In [37]:
# Run model with the best hyperparameters found for longer (10,000 epochs), 
# and with training/validation loss and MAE being recorded every 50 epochs to graph.
lr=0.0001
hidden_layers=1
neurons_per_layer=64
dropout_rate=0.5

model = create_model(hidden_layers, neurons_per_layer, dropout_rate)
optimizer = optim.Adam(model.parameters(), lr=lr)
                
training_loss_mae, validation_loss_mar = train_DNN(10000, 50, model, optimizer)

Epoch [50/10000], Training Loss: 1507220352.0, MAE: 30363
Epoch [100/10000], Training Loss: 1475094656.0, MAE: 29631
Epoch [150/10000], Training Loss: 1442358144.0, MAE: 28822
Epoch [200/10000], Training Loss: 1398189696.0, MAE: 27826
Epoch [250/10000], Training Loss: 1357984128.0, MAE: 26916
Epoch [300/10000], Training Loss: 1318979456.0, MAE: 26177
Epoch [350/10000], Training Loss: 1288840064.0, MAE: 25545
Epoch [400/10000], Training Loss: 1264314880.0, MAE: 25137
Epoch [450/10000], Training Loss: 1245840512.0, MAE: 24826
Epoch [500/10000], Training Loss: 1231648896.0, MAE: 24504
Epoch [550/10000], Training Loss: 1225777280.0, MAE: 24402
Epoch [600/10000], Training Loss: 1221485824.0, MAE: 24288
Epoch [650/10000], Training Loss: 1209425664.0, MAE: 24200
Epoch [700/10000], Training Loss: 1213701248.0, MAE: 24160
Epoch [750/10000], Training Loss: 1207500288.0, MAE: 24068
Epoch [800/10000], Training Loss: 1208375936.0, MAE: 24008
Epoch [850/10000], Training Loss: 1208333056.0, MAE: 2401

In [38]:
# Function to save plots for training and validation loss and MAE
def plot_loss_mae(training_loss_mae, validation_loss_mae, save_path):
    train_epochs, train_losses, train_mae = zip(*training_loss_mae)
    val_epochs, val_losses, val_mae = zip(*validation_loss_mae)

    # Save the Training Loss plot
    plt.plot(train_epochs, train_losses, label='Training Loss')
    plt.plot(val_epochs, val_losses, label='Validation Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(save_path + '__loss.png')
    plt.close()

    # Save the Training MAE plot
    plt.plot(train_epochs, train_mae, label='MAE (Train)')
    plt.plot(val_epochs, val_mae, label='MAE (Validation)')
    plt.title('MAE Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('MAE')
    plt.legend()
    plt.savefig(save_path + '__mae.png')
    plt.close()

In [42]:
model_save_path = 'DNN_model.pth'       # Where the model is saved
torch.save(model.state_dict(), model_save_path)

In [22]:
# Load the model (so it doesn't have to retrain every time)
model_save_path = 'DNN_model.pth'
model.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [40]:
plot_loss_mae(training_loss_mae, validation_loss_mar, 'DNN_model')

In [41]:
# Evaluate the model on the training set
train_loss, train_mae, train_mse, train_rmse, train_r2 = evaluate_model(model, X_train_tensor, y_train_tensor)
print(f"Training:\nLoss: {train_loss:.0f}\nMAE: {train_mae:.0f}\nRMSE: {train_rmse:.0f}\nR^2: {train_r2:.4f}\n")

# Evaluate the model on the validation set
val_loss, val_mae, val_mse, val_rmse, val_r2 = evaluate_model(model, X_val_tensor, y_val_tensor)
print(f"Validation:\nLoss: {val_loss:.0f}\nMAE: {val_mae:.0f}\nRMSE: {val_rmse:.0f}\nR^2: {val_r2:.4f}")

Training:
Loss: 476415776
MAE: 11023
RMSE: 21827
R^2: 0.1778

Validation:
Loss: 589718080
MAE: 11913
RMSE: 24284
R^2: 0.1497


In [ ]:
# Training:
# Loss: 465916192
# MAE: 10943
# RMSE: 21585
# R^2: 0.1960

# Validation:
# Loss: 561570880
# MAE: 11752
# RMSE: 23697
# R^2: 0.1903

### Pretty poor results still. 

# Try again with car model removed as an input variable 
# since it is a categorical variable with too many unique values.

# Training:
# Loss: 476415776
# MAE: 11023
# RMSE: 21827
# R^2: 0.1778

# Validation:
# Loss: 589718080
# MAE: 11913
# RMSE: 24284
# R^2: 0.1497

### Even worse results with car model removed. 

## Linear Regression

In [341]:
# Setup model
m = LinearRegression()
# Give input and output data
m.fit(X_train, y_train)
print('Training accuracy (the coefficient of determination): {:.3f}'.format(m.score(X_train, y_train)))

Training accuracy (the coefficient of determination): 0.983


#### Test the model on the test data

In [279]:
prediction = m.predict(X_val)

In [280]:
print('Testing accuracy by using score function: {:.3f}'.format(m.score(X_val, y_val)))
print('Testing accuracy by using r2_score meric: {:.3f}'.format(r2_score(y_val, m.predict(X_val))))

Testing accuracy by using score function: -211569106.373
Testing accuracy by using r2_score meric: -211569106.373


In [281]:
mean_absolute_error(y_val, prediction)

79078220.3044944

#### Add statsmodel for p-value

In [282]:
X_train_with_constant = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_with_constant).fit()
# Get the summary of the regression

#### Change the variables to the actual variable name

In [283]:
xnames = ['const']
xnames.extend(list(X.columns))

In [284]:
exog_names = model.model.exog_names

In [285]:
dic = dict(zip(exog_names, xnames))

In [286]:
def rename_vars(vname): 
    to_ret = vname
    for orig_vname in list(dic.keys()):
        if vname == orig_vname:
            to_ret = dic[orig_vname] 
    return to_ret

def rename_ols(reg): 
    for i in range(len(reg)): 
        reg[i] = rename_vars(reg[i])

In [287]:
rename_ols(model.model.exog_names)

In [288]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                     74.37
Date:                Thu, 16 Nov 2023   Prob (F-statistic):               0.00
Time:                        22:40:40   Log-Likelihood:                -36385.
No. Observations:                3840   AIC:                         7.611e+04
Df Residuals:                    2169   BIC:                         8.656e+04
Df Model:                        1670                                         
Covariance Type:            nonrobust                                         
                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

## Predict deal gauge using K-means

In [289]:
df = pd.read_csv('new_cars.csv')
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,...,price_change_percentage,Cylinders,Liters,Boosted,exterior_color_1,exterior_color_2,interior_color_1,interior_color_2,low mpg,high mpg
0,"""https://www.cars.com/vehicledetail/5ee3ae04-6...",2018 Hyundai Santa Fe Sport 2.4L,79497,17899,Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,...,-11.823243,4.0,2.4,False,blue,NaN,beige,NaN,21.0,27.0
1,"""https://www.cars.com/vehicledetail/c99e5035-7...",2017 RAM 1500 Express,80326,27900,This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,...,-9.956431,8.0,5.7,False,silver,NaN,black,NaN,16.0,23.0
2,"""https://www.cars.com/vehicledetail/4e79fa3a-b...",2019 Nissan Kicks SR,92959,18950,This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,...,-5.155155,4.0,1.6,False,black,orange,gray,NaN,31.0,36.0
3,"""https://www.cars.com/vehicledetail/86a0d096-3...",2019 RAM 1500 Laramie,59176,39950,Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,...,-1.414012,8.0,5.7,False,gray,NaN,black,NaN,19.0,24.0
4,"""https://www.cars.com/vehicledetail/36efbef5-f...",2015 Jeep Wrangler Unlimited Rubicon,135385,26950,This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,...,-10.016694,6.0,3.6,False,blue,NaN,black,NaN,16.0,21.0


In [290]:
# converting deal gauge to Deal
def get_deal(val):
    val = str(val).lower()
    s_list = val.split(" ")
    if "good" in s_list:
        return "Good"
    elif "great" in s_list:
        return "Great"
    elif "fair" in s_list:
        return "Fair"
    else:
        return "No Assessment Given"

df['Deal'] = df.deal_gauge.apply(lambda x: get_deal(x))
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,...,Cylinders,Liters,Boosted,exterior_color_1,exterior_color_2,interior_color_1,interior_color_2,low mpg,high mpg,Deal
0,"""https://www.cars.com/vehicledetail/5ee3ae04-6...",2018 Hyundai Santa Fe Sport 2.4L,79497,17899,Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,...,4.0,2.4,False,blue,NaN,beige,NaN,21.0,27.0,Good
1,"""https://www.cars.com/vehicledetail/c99e5035-7...",2017 RAM 1500 Express,80326,27900,This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,...,8.0,5.7,False,silver,NaN,black,NaN,16.0,23.0,Good
2,"""https://www.cars.com/vehicledetail/4e79fa3a-b...",2019 Nissan Kicks SR,92959,18950,This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,...,4.0,1.6,False,black,orange,gray,NaN,31.0,36.0,Fair
3,"""https://www.cars.com/vehicledetail/86a0d096-3...",2019 RAM 1500 Laramie,59176,39950,Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,...,8.0,5.7,False,gray,NaN,black,NaN,19.0,24.0,Great
4,"""https://www.cars.com/vehicledetail/36efbef5-f...",2015 Jeep Wrangler Unlimited Rubicon,135385,26950,This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,...,6.0,3.6,False,blue,NaN,black,NaN,16.0,21.0,Good


In [291]:
# Drop unncessary columns
columns_to_drop = ['link', 'stock_number', 'vin', 'deal_gauge', 'exterior_color', 'mpg', 'fuel_type', 'engine', 'price_history', 'vehicle_history', 'listing_title', 'interior_color']
df = df.drop(columns=columns_to_drop)

In [292]:
# Convert data types
boolean_columns = ['1-owner vehicle', 'Personal use only', 'Clean title']

for column in boolean_columns:
    df[column] = df[column].map({'Yes': True, 'No': False}).astype(bool)

In [293]:
# Check for NAs
missing_counts = df.isna().sum()
print(missing_counts[missing_counts > 0])

seller_name                  44
listed_date                1522
price_change               1522
price_change_percentage    1522
Cylinders                    73
Liters                      105
exterior_color_1            227
exterior_color_2           5989
interior_color_1            539
interior_color_2           5796
low mpg                     424
high mpg                    424
dtype: int64


In [294]:
# Drop rows with a low number of missing values
columns_with_missing_values = ['seller_name', 'Cylinders', 'Liters', 'exterior_color_1', 'low mpg', 'high mpg', 'interior_color_1']
df = df.dropna(subset=columns_with_missing_values)

# Drop exterior_color_2 since it only has 12 values
df = df.drop(columns=['exterior_color_2'])
# Drop interior_color_2 since it only has 203 values
df = df.drop(columns=['interior_color_2'])

# Drop listed_date, price_change, price_change_percentage because they account for 25% of data, 
# and they are not correlated with price in the correlation matrix.
columns_to_drop = ['listed_date', 'price_change', 'price_change_percentage']
df = df.drop(columns=columns_to_drop)

In [295]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,...,make,model,Cylinders,Liters,Boosted,exterior_color_1,interior_color_1,low mpg,high mpg,Deal
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,False,True,False,True,True,...,Hyundai,Santa Fe Sport 2.4L,4.0,2.4,False,blue,beige,21.0,27.0,Good
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,False,True,...,RAM,1500 Express,8.0,5.7,False,silver,black,16.0,23.0,Good
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,False,True,False,True,...,Nissan,Kicks SR,4.0,1.6,False,black,gray,31.0,36.0,Fair
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,True,False,False,True,True,...,RAM,1500 Laramie,8.0,5.7,False,gray,black,19.0,24.0,Great
4,135385,26950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,True,True,...,Jeep,Wrangler Unlimited Rubicon,6.0,3.6,False,blue,black,16.0,21.0,Good


In [296]:
df_map = {}  
cols = df.columns.values
print('cols:',cols)
for col in cols:
    if df[col].dtype != np.int64 and df[col].dtype != np.float64:
        temp = {}
        x = 0
        for ele in set(df[col].values.tolist()):
            if ele not in temp:
                temp[ele] = x
                x += 1

        df_map[df[col].name] = temp
        df[col] = list(map(lambda val: temp[val], df[col]))

for key, value in df_map.items():
   print(key,value)

cols: ['listing_mileage' 'primary_price' 'drivetrain' 'transmission'
 'seller_name' 'Accidents or damage' '1-owner vehicle' 'Personal use only'
 'Open recall' 'Clean title' 'year' 'make' 'model' 'Cylinders' 'Liters'
 'Boosted' 'exterior_color_1' 'interior_color_1' 'low mpg' 'high mpg'
 'Deal']
drivetrain {'Front-wheel Drive': 0, 'Unknown': 1, 'All-wheel Drive': 2, 'Four-wheel Drive': 3, 'Rear-wheel Drive': 4}
transmission {'Automatic with Tiptronic': 0, 'Automatic, 9-Spd': 1, 'Auto, 6-Spd EcoShift': 2, '8-Speed Automatic with Auto-Shift': 3, '9-Speed 948TE Automatic': 4, '8-Speed Shiftable CVT': 5, '6-Speed Shiftable Automatic w/Sport Transmission': 6, 'Automatic, 6-Spd w/SelectShift': 7, 'Auto 6-Spd w/Tptrnc & Spt': 8, '6-Speed Manual': 9, 'Auto, 8-Spd Tiptronic Spt': 10, 'Automatic 8-Speed': 11, 'Auto, 7-Spd M DCT w/Drivelogic': 12, 'Automatic, 6-Spd HD w/Overdrive': 13, '7-Speed Porsche Doppelkupplung (PDK)': 14, 'Auto, 6-Spd w/Overdrive': 15, 'Auto, 6-Spd w/Sportmatic & Active ECO 

In [297]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,...,make,model,Cylinders,Liters,Boosted,exterior_color_1,interior_color_1,low mpg,high mpg,Deal
0,79497,17899,0,38,99,0,1,0,1,1,...,19,836,4.0,2.4,0,4,4,21.0,27.0,0
1,80326,27900,3,22,110,0,0,1,0,1,...,9,67,8.0,5.7,0,0,8,16.0,23.0,0
2,92959,18950,0,51,110,0,0,1,0,1,...,15,966,4.0,1.6,0,10,1,31.0,36.0,1
3,59176,39950,3,22,110,1,0,0,1,1,...,9,85,8.0,5.7,0,1,8,19.0,24.0,3
4,135385,26950,3,22,110,0,0,1,1,1,...,33,364,6.0,3.6,0,4,8,16.0,21.0,0


In [298]:
# apply the k-means model
x = np.array(df.drop(['Deal'], 1).astype(float))
x = preprocessing.scale(x)

clf = KMeans(n_clusters=3)
clf.fit(x)

y = np.array(df['Deal'])

C:\Users\Evan\AppData\Local\Temp/ipykernel_23896/1314405606.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = np.array(df.drop(['Deal'], 1).astype(float))


In [299]:
# calulate accuracy
correct = 0
for i in range(len(x)):
    predict_data = np.array(x[i].astype(float))
    predict_data = predict_data.reshape(-1, len(predict_data))
    predict = clf.predict(predict_data)
    # print(predict[0], y[i])
    if predict[0] == y[i]:
        correct += 1

print(f"accuracy :{correct * 1.0 / len(x)}")

accuracy :0.3318058737762966
